In [1]:
import requests
import pandas as pd
from datetime import date
import datetime
from datetime import datetime
import calendar
from bs4 import BeautifulSoup
import plotly.express as px

In [2]:
def Agro_price(clave_producto, central_abastos):
    appended_data = []
    for i in range(1998,datetime.now().year+1):
        response = requests.get('http://economia-sniim.gob.mx/Consolidados.asp?dqdia=1&dqmes=01&dqanio={}&aqdia=31&aqmes=12&aqanio={}&Prod={}&Edo=&punto={}&det=S'.format(i,i,clave_producto,central_abastos), verify = False)
        soup = BeautifulSoup(response.content,'html.parser')

        section = soup.find_all('table')
        links = section[11].findChildren("tr" , recursive=False)[2:]

        upperframe=[]
        frame =[]

        for i in links:
            aux = i.find_all('td')
            Fecha = aux[0].text
            Min = aux[1].text
            Max = aux[2].text
            Frec = aux[3].text
            Origen = aux[4].text
    
            frame.append((Fecha,Min, Max,Frec,Origen))

        
        upperframe.extend(frame)
        appended_data.extend(upperframe)
    database= pd.DataFrame(appended_data, columns=['Fecha','Min', 'Max','Frec','Origen'])
    database['Fecha'] = pd.to_datetime(database['Fecha'], dayfirst=True)
    database['Dia_sem'] =[calendar.day_name[i.weekday()] for i in database['Fecha']]
    return database

In [ ]:
Aguacate_Hass = Agro_price(133, 210)
Aguacate_Hass

In [19]:
import locale
locale.setlocale(locale.LC_TIME, 'es_ES')

'es_ES'

In [172]:
aux= Aguacate_Hass.loc[Aguacate_Hass['Fecha']>= '01-01-2020'].reset_index(drop = True)
fig = px.line(aux, x="Fecha", y='Frec',
              title='Precio del Aguacate')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b/%Y")
fig.update_layout(
    yaxis_title="Precio promedio")
fig.show()

In [130]:
aux1 = pd.DataFrame(((Aguacate_Hass.groupby(["year",'Origen']).count()['Fecha'])/(Aguacate_Hass.groupby("year").count()['Fecha'])*100).round().astype(int)).reset_index()
fig = px.bar(aux1[aux1['year'] >= 2010], x="Fecha", y="year", color="Origen",
             color_discrete_sequence=['#003f5c', '#7a5195','#ef5675', '#ffa600'],
             title="Estado de Origen - Aguacate Hass", text_auto = True,orientation='h')
fig.layout.xaxis.ticksuffix="%"
fig.update_layout(
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.95],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        visible = False,
        domain=[0, 0.60],
    ),
   
    legend=dict(x=0.6, y=0.95, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='#FFFFFF',
    plot_bgcolor='#FFFFFF',
    xaxis_title="",
    yaxis_title="Año",uniformtext_minsize=8, uniformtext_mode='hide'
)
fig.update_yaxes(autorange="reversed")
fig.show()

In [175]:
aux2= aux[aux['Same_price']==0]
aux2


,Fecha,Min,Max,Frec,Origen,Dia_sem,year,Same_price
0,2020-01-02,30.00,40.00,40.00,Sin,Thursday,2020,0
6,2020-01-10,30.00,40.00,35.00,Sin,Friday,2020,0
36,2020-02-24,30.00,40.00,40.00,Sin,Monday,2020,0
38,2020-02-26,30.00,38.00,38.00,Sin,Wednesday,2020,0
41,2020-03-02,30.00,35.00,35.00,Sin,Monday,2020,0
...,...,...,...,...,...,...,...,...
690,2023-01-11,45.00,50.00,50.00,Sin,Wednesday,2023,0
702,2023-01-27,40.00,45.00,45.00,Sin,Friday,2023,0
707,2023-02-03,40.00,45.00,40.00,Sin,Friday,2023,0
709,2023-02-08,40.00,45.00,45.00,Sin,Wednesday,2023,0


In [176]:
fig = px.line(aux2, x="Fecha", y='Frec',
              title='Precio del Aguacate')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b/%Y")
fig.update_layout(
    yaxis_title="Precio promedio")
fig.show()